# Modelo    

In [3]:
from dataclasses import dataclass
from typing import Any, List
import numpy as np

@dataclass
class Emotions:
    name : str
    quantity : int

@dataclass
class Point:
    x : float
    y : float

@dataclass
class BoundingBox:
    origin : Point
    end: Point


class Genre:
    male = 'MALE'
    female = 'FEMALE'

    def __init__(self, genre : str):
        self.genre = genre

    @staticmethod
    def female(cls):
        return Genre(Genre.female)
    
    @staticmethod
    def male(cls):
        return Genre(Genre.male)
    
    def isMale(self):
        return self.genre == Genre.male

    def isFemale(self):
        return self.genre == Genre.female

@dataclass
class Person:
    age : str
    genre : Genre
    emotions : List[Emotions]
    bounding_box : BoundingBox
    image : np.ndarray

@dataclass
class FaceDetectorResult:
    image : np.ndarray 
    bounding_box : BoundingBox

@dataclass
class FaceComparatorResult:
    similarity : float

# Aplicación

In [4]:
from abc import ABC, abstractmethod
from typing import List

class FaceDetector(ABC):
    @abstractmethod
    def detect(self, image : np.nd) -> List[FaceDetectorResult]:
        pass

class FaceQualifier(ABC):
    @abstractmethod
    def qualify(self, face_detector_result : FaceDetectorResult) -> List[Person]:
        pass

class FaceComparator(ABC):
    @abstractmethod
    def qualify(self, first_face : FaceDetectorResult, second_face: FaceDetectorResult) -> FaceComparatorResult:
        pass

# Infraestructura


In [20]:
class ImageUtils:
    @staticmethod
    def crop(image : np.array, bounding_box : BoundingBox) -> np.ndarray:
        return image[bounding_box.origin.x:bounding_box.end.x, bounding_box.origin.y:bounding_box.end.y]

In [27]:
import os
import cv2
from ultralytics import YOLO
from retinaface import RetinaFace
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np

class ViolaJonesFaceDetector(FaceDetector):
    cascPathface = os.path.dirname(
        cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"


    def detect(self, image : np.ndarray) -> List[FaceDetectorResult]:
        faceCascade = cv2.CascadeClassifier(self.cascPathface)
        gray = self._convert_image_to_gray(image)
        faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
        return self._convert_to_face_detector_result(image, faces)
        
    
    def _convert_to_face_detector_result(self, image, faces) -> List[FaceDetectorResult]:
        bounding_boxes = [BoundingBox(Point(x, y), Point(x+w, y+h)) for (x, y, w, h) in faces]
        return [FaceDetectorResult(ImageUtils.crop(image, bounding_box), bounding_box) for bounding_box in bounding_boxes]
    
    def _convert_image_to_gray(self, image : np.ndarray) -> np.ndarray:
        import cv2
        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

class RetinafaceFaceDetector(FaceDetector):
    def detect(self, image : np.ndarray) -> List[FaceDetectorResult]:
        faces = RetinaFace.detect_faces(image);
        return self._convert_to_face_detector_result(image, [faces[key]['facial_area'] for key in faces.keys()])
    
    def _convert_to_face_detector_result(self, image, faces) -> List[FaceDetectorResult]:
        bounding_boxes = [BoundingBox(Point(x, y), Point(w, h)) for (x, y, w, h) in faces]
        return [FaceDetectorResult(ImageUtils.crop(image, bounding_box), bounding_box) for bounding_box in bounding_boxes]

class MediaPipeFaceDetector(FaceDetector):
    def detect(self, image : np.ndarray) -> List[FaceDetectorResult]:
        BaseOptions = mp.tasks.BaseOptions
        FaceDetector = mp.tasks.vision.FaceDetector
        FaceDetectorOptions = mp.tasks.vision.FaceDetectorOptions
        VisionRunningMode = mp.tasks.vision.RunningMode

        options = FaceDetectorOptions(
            base_options=BaseOptions(model_asset_path='./blaze_face_short_range.tflite'),
            running_mode=VisionRunningMode.IMAGE)

        with FaceDetector.create_from_options(options) as detector:
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
            face_detector_result = detector.detect(mp_image)
            return self._convert_to_face_detector_result(image, [(x.bounding_box.origin_x, x.bounding_box.origin_y, x.bounding_box.width, x.bounding_box.height) for x in face_detector_result.detections])
        
    def _convert_to_face_detector_result(self, image, faces) -> List[FaceDetectorResult]:
        bounding_boxes = [BoundingBox(Point(x, y), Point(x + w, y + h)) for (x, y, w, h) in faces]
        return [FaceDetectorResult(ImageUtils.crop(image, bounding_box), bounding_box) for bounding_box in bounding_boxes]
    

In [30]:
from typing import List


class MockFaceQualifier(FaceQualifier):
    def qualify(self, face_detector_result : FaceDetectorResult) -> List[Person]:
        return Person(
            age=43, 
            genre=Genre.male(), 
            emotions=[Emotions('happy', 0.8)], 
            bounding_box=face_detector_result.bounding_box, 
            image=face_detector_result.image)

class DeepfaceFaceQualifier(FaceQualifier):
    def qualify(self, face_detector_result: FaceDetectorResult) -> List[Person]:
        pass

In [ ]:

class MockFaceComparator(FaceComparator):
    def qualify(self, first_face : FaceDetectorResult, second_face: FaceDetectorResult) -> FaceComparatorResult:
        return FaceComparatorResult(0.8)
    
class DeepfaceFaceComparator(FaceComparator):
    def qualify(self, first_face : FaceDetectorResult, second_face: FaceDetectorResult) -> FaceComparatorResult:
        pass

# CV2 INTERFACE

In [ ]:
class FaceQualificationDisplay(ABC):
    @abstractmethod
    def display(self) -> None:
        pass

class OpenCVFaceQualificationDisplay(ABC):
    def __init__(self, people : List[Person], frame : np.array) -> None:
        self.people = people
    
    def display(self):
        pass
    
    def paint_interface(self, person : Person, frame: np.ndarray):
        pass

In [ ]:
class AuthenticationScreen(ABC):
    @abstractmethod
    def display(self) -> None:
        pass

class LockAuthenticationScreen(AuthenticationScreen):
    def display(self) -> None:
        pass

class AccessGrantedAuthenticationScreen(AuthenticationScreen):
    def display(self) -> None:
        pass

class AccessDeniedAuthenticationScreen(AuthenticationScreen):
    def display(self) -> None:
        pass

@dataclass
class State:
    screen : AuthenticationScreen

class AuthenticationStateMachine(ABC):
    pass


In [ ]:
import cv2 as cv

video = cv.VideoCapture(0)

while True:
    ret, frame = video.read()
    face_detector = MediaPipeFaceDetector()
    faces = face_detector.detect(frame)
    for x in faces:
        cv.rectangle(frame, (x.bounding_box.origin.x, x.bounding_box.origin.y), (x.bounding_box.end.x, x.bounding_box.end.y), (255, 0, 0), 2)
    
    for x in faces:
        qualifier = DeepfaceFaceQualifier()
        qualification = qualifier.qualify(x)
        print(qualification)
    cv.imshow('Video', frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
